### **Testing various NN's on time series**

##### Imports

In [ ]:
# N-BEATS, PatchTST
from neuralforecast.models import NBEATS, PatchTST
from neuralforecast.losses.pytorch import HuberLoss
from neuralforecast.core import NeuralForecast

import joblib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, make_scorer
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import RandomizedSearchCV

# Hyperparameter tuning
import optuna

# N-HiTS
from darts import TimeSeries
from darts.models import NHiTSModel
from torch.nn import MSELoss

# DeepAR
from gluonts.dataset.pandas import PandasDataset
from gluonts.torch.model.deepar import DeepAREstimator


##### Standard scaling

In [ ]:
def standard_scaling(x):
    mean = np.mean(np.abs(x))
    s = np.std(x)

    return (x - mean)/s

def standard_unscaling(original, scaled):
    mean = np.mean(np.abs(original))
    s = np.std(original)

    return (scaled * s) + mean

# Scaler, that scales data according to other data
def standard_scaling_transform(original, to_scale):
    mean = np.mean(np.abs(original))
    s = np.std(original)

    return (to_scale - mean)/s

### **1** N-BEATS

#### **1.1** Aquifer data

In [ ]:
# Read the dataset
aquifer_by_stations = joblib.load('aquifer_by_stations.joblib')

##### Quick test

In [ ]:
Y_df = aquifer_by_stations[1010]

In [ ]:
Y_df = Y_df.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

In [ ]:
Y_df = Y_df[['ds', 'y', 'unique_id']]

In [ ]:
horizon = 12
val_size = 12
test_size = 12

nbeats = NBEATS(h=horizon, input_size=3*horizon, loss=HuberLoss(), devices=1, accelerator='cuda')

nf = NeuralForecast(models=[nbeats], freq='D')
nbeats_forecasts_df = nf.cross_validation(df=Y_df[:-12], val_size=val_size, test_size=test_size, n_windows=None, verbose=True)


nbeats_forecasts_df.head()

In [ ]:
nbeats_forecasts_df

In [ ]:
nbeats_forecasts_df.iloc[:12]

In [ ]:
# Plot the results
plt.figure(figsize=(8, 4))
plt.plot(Y_df['ds'][-200:], Y_df['y'][-200:], color='royalblue', label='true data')
plt.plot(Y_df['ds'][-12:], nbeats_forecasts_df['NBEATS'].iloc[:12], color='tomato', label='prediction')
plt.grid()
plt.legend()
plt.show()

##### Testing on multiple stations

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Remove the last 5 days
# This is done to enable direct comparison to the randomforest,
# there the 5 days are removed because of the weather forecast generation
for aquifer in aquifers_list:
    aquifer_by_stations[aquifer] = aquifer_by_stations[aquifer][:-5]

In [ ]:
'''{'input_size': 15,
 'n_harmonics': 5,
 'n_polynomials': 5,
 'scaler_type': 'robust',
 'learning_rate': 0.001,
 'max_steps': 25,
 'val_size': 10,
 'n_blocks_season': 3,
 'n_blocks_trend': 3,
 'n_blocks_ident': 1,
 'mlp_units': 128,
 'num_hidden': 1}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast
val_size = 2*horizon

models = [NBEATS(h=horizon, 
                 loss=HuberLoss(),
                 accelerator='cuda',
                 input_size=3*horizon,
                 n_harmonics=5,
                 n_polynomials=5,
                 scaler_type='robust',
                 learning_rate=0.001,
                 max_steps=25,
                 n_blocks=[3, 3, 1],
                 mlp_units=[[128, 128]],
                 devices=[0],
                 logger=False)]

model = NeuralForecast(models=models, freq='D')

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

# Dictionary for storing the predictions
predictions_by_stations = {key: [] for key in aquifer_by_stations}

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

    # Only keep these 3 columns
    y = y[['ds', 'y', 'unique_id']]

    # Fit the model
    model.fit(y[:-day_len], val_size=val_size)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Predict
        forecast = model.predict(df=y[:-i], verbose=0)

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['NBEATS'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-365:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Store the predictions to the dictionary
    predictions_by_stations[aquifer] = predictions

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-200:], aquifer_by_stations[aquifer]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions[2], color="tomato", label="forecast")
plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions_by_stations[85064][2], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-beats/n-beats-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-beats/n-beats-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-beats/n-beats-ground-water-r2-stations.joblib')

In [ ]:
# Save the dictionary with predictions
joblib.dump(predictions_by_stations, '../reports/n-beats/n-beats-ground-water-predictions.joblib')

##### Quick test #2

In [ ]:
h = 5

train = aquifer_by_stations[1010][:-h]
train = train.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})
train = train[['ds', 'y', 'unique_id']]

In [ ]:
train

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS 
from neuralforecast.losses.pytorch import DistributionLoss

models = [NBEATS(h=h,input_size=3*h,
                 loss=HuberLoss(),
                 max_steps=200,
                 scaler_type='standard',
                 accelerator='cuda')]


model = NeuralForecast(models=models, freq='D')
model.fit(train, val_size=h)


p =  model.predict(train)
p

In [ ]:
# Plot the results
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[1010]['date'][-20:], aquifer_by_stations[1010]['altitude_diff'][-20:], color='royalblue', label='true data')
plt.plot(p['ds'].iloc[:h], p['NBEATS'].iloc[:h], color='tomato', label='prediction')
plt.grid()
plt.legend()
plt.show()

##### Hyperparameter tuning

<sub>We do hyperparameter tuning by choosing 4 stations from the ones we are going to test. We leave the last 200 days for testing. We use the last 100 days of the remaining dataset for the validation. On the validation we test the parameters</sub>

In [ ]:
#%pip install optuna

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 200

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_size = trial.suggest_categorical('input_size', [horizon, 2*horizon, 3* horizon, 4*horizon])
    
    n_harmonics = trial.suggest_int('n_harmonics', 1, 5)
    n_polynomials = trial.suggest_int('n_polynomials', 1, 5)
    
    scaler_type = trial.suggest_categorical('scaler_type', ['standard', 'robust'])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])

    max_steps = trial.suggest_categorical('max_steps', [10, 25, 50, 10, 200])

    validation_size = trial.suggest_categorical('val_size', [horizon, 2*horizon, 3*horizon])

    n_blocks_season = trial.suggest_int('n_blocks_season', 1, 3)
    n_blocks_trend = trial.suggest_int('n_blocks_trend', 1, 3)
    n_blocks_identity = trial.suggest_int('n_blocks_ident', 1, 3)
    
    mlp_units_n = trial.suggest_categorical('mlp_units', [32, 64, 128, 256, 512])
    num_hidden = trial.suggest_int('num_hidden', 1, 3)
    
    n_blocks = [n_blocks_season, n_blocks_trend, n_blocks_identity]
    mlp_units=[[mlp_units_n, mlp_units_n]]*num_hidden

    models = [NBEATS(h=horizon,input_size=input_size,
                 loss=HuberLoss(),
                 max_steps=max_steps,
                 learning_rate=learning_rate,
                 n_harmonics=n_harmonics,
                 n_polynomials=n_polynomials,
                 scaler_type=scaler_type,
                 mlp_units=mlp_units,
                 n_blocks=n_blocks,
                 accelerator='cuda',
                 logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

        # Only keep these 3 columns
        y = y[['ds', 'y', 'unique_id']]

        # Fit the model
        model.fit(y[:-day_len], val_size=validation_size)

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i], verbose=0)

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['NBEATS'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

#### **1.2** Surface water data

In [ ]:
# Read the dataset
watercourse_by_stations = joblib.load('../data/interim/watercourse_by_stations.joblib')

##### Hyperparemeter tuning

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 200

In [ ]:
station_list = [4270, 4570, 4515, 6068]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_size = trial.suggest_categorical('input_size', [horizon, 2*horizon, 3* horizon, 4*horizon])
    
    n_harmonics = trial.suggest_int('n_harmonics', 1, 5)
    n_polynomials = trial.suggest_int('n_polynomials', 1, 5)
    
    scaler_type = trial.suggest_categorical('scaler_type', ['standard', 'robust'])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])

    max_steps = trial.suggest_categorical('max_steps', [10, 25, 50, 10, 200])

    validation_size = trial.suggest_categorical('val_size', [horizon, 2*horizon, 3*horizon])

    n_blocks_season = trial.suggest_int('n_blocks_season', 1, 3)
    n_blocks_trend = trial.suggest_int('n_blocks_trend', 1, 3)
    n_blocks_identity = trial.suggest_int('n_blocks_ident', 1, 3)
    
    mlp_units_n = trial.suggest_categorical('mlp_units', [32, 64, 128, 256, 512])
    num_hidden = trial.suggest_int('num_hidden', 1, 3)
    
    n_blocks = [n_blocks_season, n_blocks_trend, n_blocks_identity]
    mlp_units=[[mlp_units_n, mlp_units_n]]*num_hidden

    models = [NBEATS(h=horizon,input_size=input_size,
                 loss=HuberLoss(),
                 max_steps=max_steps,
                 learning_rate=learning_rate,
                 n_harmonics=n_harmonics,
                 n_polynomials=n_polynomials,
                 scaler_type=scaler_type,
                 mlp_units=mlp_units,
                 n_blocks=n_blocks,
                 accelerator='cuda',
                 logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for station in station_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = watercourse_by_stations[station][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

        # Only keep these 3 columns
        y = y[['ds', 'y', 'unique_id']]

        # Fit the model
        model.fit(y[:-day_len], val_size=validation_size)

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i], verbose=0)

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['NBEATS'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing on multiple stations

In [ ]:
# List of station used for testing
station_list = ['2530', '2620', '4200', '4230', '4270', '4515', '4520', '4570', '4575', '5040', '5078', '5330', '5425', '5500', '6060', '6068', '6200', '6220', '6300', '6340', '8454', '8565']

In [ ]:
# Cast the stations to int
for i in range(len(station_list)):
    station_list[i] = int(station_list[i])

In [ ]:
'''{'input_size': 10,
 'n_harmonics': 2,
 'n_polynomials': 4,
 'scaler_type': 'robust',
 'learning_rate': 0.01,
 'max_steps': 50,
 'val_size': 10,
 'n_blocks_season': 1,
 'n_blocks_trend': 1,
 'n_blocks_ident': 3,
 'mlp_units': 32,
 'num_hidden': 2}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 200 # number of days to forecast
val_size = 2*horizon

models = [NBEATS(h=horizon, 
                 loss=HuberLoss(),
                 accelerator='cuda',
                 input_size=2*horizon,
                 n_harmonics=2,
                 n_polynomials=4,
                 scaler_type='robust',
                 learning_rate=0.01,
                 max_steps=50,
                 n_blocks=[1, 1, 3],
                 mlp_units=[[32, 32], [32, 32]],
                 logger=False)]

model = NeuralForecast(models=models, freq='D')

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for station in station_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = watercourse_by_stations[station]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

    # Only keep these 3 columns
    y = y[['ds', 'y', 'unique_id']]

    # Fit the model
    model.fit(y[:-day_len], val_size=horizon)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Predict
        forecast = model.predict(df=y[:-i], verbose=0)

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['NBEATS'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-200:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(watercourse_by_stations[station]['level_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(watercourse_by_stations[station]['date'][-200:], watercourse_by_stations[station]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(watercourse_by_stations[station]['date'][-day_len:], predictions[0], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-beats/n-beats-surface-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-beats/n-beats-surface-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(station_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-beats/n-beats-surface-water-r2-stations.joblib')

### **2** N-HiTS

##### Imports

In [ ]:
%pip install darts

#### **2.1** Aquifer data

##### Quick test

In [ ]:
horizon = 50

In [ ]:
# Set the variable target as altitude_diff
data = aquifer_by_stations[1010][:-horizon]

In [ ]:
# Assuming 'data' is a pandas DataFrame with a datetime index and one or more columns
target = TimeSeries.from_dataframe(data, time_col='date', value_cols='altitude_diff')


In [ ]:
# Set the model parameters
model = NHiTSModel(
    input_chunk_length=6,
    output_chunk_length=6,
    num_blocks=2,
    n_epochs=50
    #pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]}
)
# Fit the model
model.fit(target) #, pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]})

# Make predictions
pred = model.predict(horizon)

In [ ]:
pred.values()[0][0]

In [ ]:
predictions=[]
for value in pred.values():
    predictions.append(value[0])

In [ ]:
predictions

In [ ]:
# Visualise the result
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[1010]['date'][-200:], aquifer_by_stations[1010]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(aquifer_by_stations[1010]['date'][-horizon:], predictions, color="tomato", label="prediction")
plt.legend()
plt.grid(True)
plt.show()

##### Hyperparameter tuning

In [ ]:
# Define the horizon, day_len (number of predicted days), test_len (number of days used for final testing)
horizon = 5
day_len = 100
test_len = 200

In [ ]:
# Stations to test
aquifers_list = [85065, 85064]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_chunk_length = trial.suggest_int('input_chunk_length', 5, 70)
    output_chunk_length = trial.suggest_int('output_chunk_length', 1, 10)
    num_stacks = trial.suggest_int('num_stacks', 1, 4)
    num_blocks = trial.suggest_int('num_blocks', 1, 3)
    num_layers = trial.suggest_int('num_layers', 2, 5)
    layer_widths = trial.suggest_categorical('layer_widths', [64, 128, 256, 512])
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-2, 1e-3, 1e-4])
    n_epochs = trial.suggest_int('n_epochs', 10, 200)

    model = NHiTSModel(input_chunk_length=input_chunk_length,
                     output_chunk_length=output_chunk_length,
                     num_stacks=num_stacks,
                     num_blocks=num_blocks,
                     num_layers=num_layers,
                     layer_widths=layer_widths,
                     dropout=dropout,
                     optimizer_kwargs={'lr': learning_rate},
                     n_epochs=n_epochs,
                     pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]})
    


    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Change to TimeSeries format (required by the library)
        y = TimeSeries.from_dataframe(y, time_col='date', value_cols='altitude_diff')

        # Fit the model
        model.fit(y[:-day_len])

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(n=horizon, series=y[:-i])


            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast.values()[i][0])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-(day_len+test_len):-test_len], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Test on multiple stations

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
'''{'input_chunk_length': 46,
 'output_chunk_length': 8,
 'num_stacks': 3,
 'num_blocks': 2,
 'num_layers': 4,
 'layer_widths': 512,
 'dropout': 0.2,
 'learning_rate': 0.0001,
 'n_epochs': 89}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 200 # number of days to forecast

# Set the model parameters
model = NHiTSModel(
    input_chunk_length=46,
    output_chunk_length=8,
    num_blocks=2,
    num_stacks=3,
    num_layers=4,
    layer_widths=512,
    dropout=0.2,
    n_epochs=89,
    optimizer_kwargs={'lr': 1e-4},
    pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]}
)


# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer]

    # Change the format to TimeSeries
    y = TimeSeries.from_dataframe(y, time_col='date', value_cols='altitude_diff')
    

    # Fit the model
    model.fit(y[:-day_len])

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Make predictions
        forecast = model.predict(n=horizon, series=y[:-i])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast.values()[i][0])
    
    # Clean up the results
    predictions[0] = predictions[0][-200:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-200:], aquifer_by_stations[aquifer]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions[3], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-hits/n-hits-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-hits/n-hits-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-hits/n-hits-ground-water-r2-stations.joblib')

#### **2.2** Watercourse data

##### Hyperparameter tuning

In [ ]:
# Define the horizon, day_len (number of predicted days), test_len (number of days used for final testing)
horizon = 5
day_len = 100
test_len = 200

In [ ]:
# Stations to test
station_list = [4270, 4570, 4515, 6068]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_chunk_length = trial.suggest_int('input_chunk_length', 5, 70)
    output_chunk_length = trial.suggest_int('output_chunk_length', 1, 10)
    num_stacks = trial.suggest_int('num_stacks', 1, 4)
    num_blocks = trial.suggest_int('num_blocks', 1, 3)
    num_layers = trial.suggest_int('num_layers', 2, 5)
    layer_widths = trial.suggest_categorical('layer_widths', [64, 128, 256, 512])
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-2, 1e-3, 1e-4])
    n_epochs = trial.suggest_int('n_epochs', 10, 200)

    model = NHiTSModel(input_chunk_length=input_chunk_length,
                     output_chunk_length=output_chunk_length,
                     num_stacks=num_stacks,
                     num_blocks=num_blocks,
                     num_layers=num_layers,
                     layer_widths=layer_widths,
                     dropout=dropout,
                     optimizer_kwargs={'lr': learning_rate},
                     n_epochs=n_epochs,
                     pl_trainer_kwargs={'logger': False, "accelerator": "gpu", "devices": [0]})
    


    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for station in station_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = watercourse_by_stations[station][:-test_len]

        # Change to TimeSeries format (required by the library)
        y = TimeSeries.from_dataframe(y, time_col='date', value_cols='level_diff')

        # Fit the model
        model.fit(y[:-day_len])

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(n=horizon, series=y[:-i])


            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast.values()[i][0])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(watercourse_by_stations[station]['level_diff'][-(day_len+test_len):-test_len], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    print(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Test on multiple stations

In [ ]:
# List of station used for testing
station_list = ['2530', '2620', '4200', '4230', '4270', '4515', '4520', '4570', '4575', '5040', '5078', '5330', '5425', '5500', '6060', '6068', '6200', '6220', '6300', '6340', '8454', '8565']

In [ ]:
# Cast the stations to int
for i in range(len(station_list)):
    station_list[i] = int(station_list[i])

In [ ]:
horizon = 5 # prediction horizon
day_len = 200 # number of days to forecast

# Set the model parameters
model = NHiTSModel(
    input_chunk_length=3*horizon,
    output_chunk_length=3*horizon,
    num_blocks=2,
    n_epochs=50,
)


# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for station in station_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = watercourse_by_stations[station]

    # Change the format to TimeSeries
    y = TimeSeries.from_dataframe(y, time_col='date', value_cols='level_diff')

    # Fit the model
    model.fit(y[:-day_len])

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Make predictions
        forecast = model.predict(n=horizon, series=y[:-i])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast.values()[i][0])
    
    # Clean up the results
    predictions[0] = predictions[0][-200:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(watercourse_by_stations[station]['level_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(watercourse_by_stations[station]['date'][-200:], watercourse_by_stations[station]['level_diff'][-200:], color="royalblue", label="true data")
plt.plot(watercourse_by_stations[station]['date'][-day_len:], predictions[0], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-hits/n-hits-surface-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-hits/n-hits-surface-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(station_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-hits/n-hits-surface-water-r2-stations.joblib')

### **3** PatchTST

#### **3.1** Aquifer data

##### Quick test

In [ ]:
h = 10

train = aquifer_by_stations[1010][:-h]
train = train.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})
train = train[['ds', 'y', 'unique_id']]

In [ ]:
{'input_size': 55,
 'encoder_layers': 2,
 'n_heads': 2,
 'hidden_size': 64,
 'linear_hidden_size': 512,
 'dropout': 0.2,
 'fc_dropout': 0.1,
 'head_dropout': 0.1,
 'attn_dropout': 0.2,
 'patch_len': 2,
 'stride': 3,
 'revin': True,
 'learning_rate': 1,
 'max_steps': 1988}

In [ ]:
model = PatchTST(h=h,
                 input_size=55,
                 encoder_layers=4,
                 n_heads=8,
                 hidden_size=64,
                 linear_hidden_size=512,
                 dropout=0.2,
                 fc_dropout=0.1,
                 head_dropout=0.1,
                 attn_dropout=0.2,
                 patch_len=32,
                 stride=24,
                 revin=True,
                 learning_rate=1e-1,
                 max_steps=1988,
                 logger=False)

nf = NeuralForecast(
    models=[model],
    freq='D'
)
nf.fit(df=train, val_size=h)
forecasts = nf.predict()

In [ ]:
# Plot the results
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[1010]['date'][-20:], aquifer_by_stations[1010]['altitude_diff'][-20:], color='royalblue', label='true data')
plt.plot(forecasts['ds'].iloc[:h], forecasts['PatchTST'].iloc[:h], color='tomato', label='prediction')
plt.grid()
plt.legend()
plt.show()

##### Hyperparameter tuning

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 200

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_size = trial.suggest_int('input_size', 5, 100)
    encoder_layers = trial.suggest_int('encoder_layers', 1, 4)
    encoder_layers = 2*encoder_layers
    n_heads = trial.suggest_int('n_heads', 1, 3)
    if n_heads == 3:
        n_heads = 4
    n_heads = 8*n_heads
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])
    linear_hidden_size = trial.suggest_categorical('linear_hidden_size', [128, 256, 512])
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2])
    fc_dropout = trial.suggest_categorical('fc_dropout', [0.1, 0.2])
    head_dropout = trial.suggest_categorical('head_dropout', [0.1, 0.2])
    attn_dropout = trial.suggest_categorical('attn_dropout', [0.1, 0.2])
    patch_len = trial.suggest_int('patch_len', 1, 4)
    patch_len = 16*patch_len
    stride = trial.suggest_int('stride', 1, 4)
    stride = 8*stride
    revin = trial.suggest_categorical('revin', [True, False])
    learning_rate = trial.suggest_int('learning_rate', 1, 5)
    learning_rate = 10**(-learning_rate)
    max_steps = trial.suggest_int('max_steps', 100, 2000)

    models = [PatchTST(h=horizon,
                       input_size=input_size,
                       encoder_layers=encoder_layers,
                       n_heads=n_heads,
                       hidden_size=hidden_size,
                       linear_hidden_size=linear_hidden_size,
                       dropout=dropout,
                       fc_dropout=fc_dropout,
                       head_dropout=head_dropout,
                       attn_dropout=attn_dropout,
                       patch_len=patch_len,
                       stride=stride,
                       revin=revin,
                       learning_rate=learning_rate,
                       max_steps=max_steps,
                       logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

        # Only keep these 3 columns
        y = y[['ds', 'y', 'unique_id']]

        # Fit the model
        model.fit(y[:-day_len])

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i])

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['PatchTST'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing multiple stations

In [ ]:
# Read the dataset
aquifer_by_stations = joblib.load('aquifer_by_stations.joblib')

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
# Remove the last 5 days
# This is done to enable direct comparison to the randomforest,
# there the 5 days are removed because of the weather forecast generation
for aquifer in aquifers_list:
    aquifer_by_stations[aquifer] = aquifer_by_stations[aquifer][:-5]

In [ ]:
'''{'input_size': 71,
 'encoder_layers': 6,
 'n_heads': 2,
 'hidden_size': 64,
 'linear_hidden_size': 512,
 'dropout': 0.2,
 'fc_dropout': 0.1,
 'head_dropout': 0.1,
 'attn_dropout': 0.2,
 'patch_len': 1,
 'stride': 1,
 'revin': True,
 'learning_rate': 3,
 'max_steps': 1323}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast
val_size = horizon

model = PatchTST(h=horizon,
                 input_size=71,
                 encoder_layers=12,
                 n_heads=16,
                 hidden_size=64,
                 linear_hidden_size=512,
                 dropout=0.2,
                 fc_dropout=0.1,
                 head_dropout=0.1,
                 attn_dropout=0.2,
                 patch_len=16,
                 stride=8,
                 revin=True,
                 learning_rate=1e-3,
                 max_steps=1323,
                 logger=False)

nf = NeuralForecast(
    models=[model],
    freq='D'
)

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

# Dictionary for the predictions from all of the different aquifers
predictions_by_stations = {key: [] for key in aquifers_list}

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'altitude_diff':'y', 'station_id':'unique_id'})

    # Only keep these 3 columns
    y = y[['ds', 'y', 'unique_id']]
    y_train = y[:-day_len]

    # Scale the training set
    y_train['y'] = standard_scaling(y['y'][:-day_len])

    # Fit the model
    nf.fit(y_train, val_size=val_size)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Scale the testing set
        y_test = y[:-i]
        y_test['y'] = standard_scaling_transform(original=y['y'][:-day_len], to_scale=y['y'][:-i])

        # Predict
        forecast = nf.predict(df=y_test)

        # Unscale the prediction
        forecast['PatchTST'] = standard_unscaling(y['y'][:-day_len], forecast['PatchTST'])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['PatchTST'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Add the predictions to the dictionary
    predictions_by_stations[aquifer] = predictions

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
#plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], color="royalblue", label="true data")
#plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions[2], color="tomato", label="forecast")
#plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions_by_stations[85064][2], color="tomato", label="forecast")
#plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], y['y'][-day_len:], color="tomato", label="forecast")
plt.plot(standard_unscaling(y['y'][:-day_len], y_train['y']))
#plt.plot(y['y'][:-day_len])
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/patchtst/patchtst-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/patchtst/patchtst-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/patchtst/patchtst-ground-water-r2-stations.joblib')

In [ ]:
# Save the dictionary with predictions
joblib.dump(predictions_by_stations, '../reports/patchtst/patchtst-ground-water-predictions.joblib')

#### **3.2** Watercourse data

##### Hyperparameter tuning

In [ ]:
# Define the horizon and the day_len
horizon = 5
day_len = 100
test_len = 200

In [ ]:
# Stations to test
station_list = [4270, 4570, 4515, 6068]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    input_size = trial.suggest_int('input_size', 5, 100)
    encoder_layers = trial.suggest_int('encoder_layers', 1, 4)
    encoder_layers = 2*encoder_layers
    n_heads = trial.suggest_int('n_heads', 1, 3)
    if n_heads == 3:
        n_heads = 4
    n_heads = 8*n_heads
    hidden_size = trial.suggest_categorical('hidden_size', [64, 128, 256])
    linear_hidden_size = trial.suggest_categorical('linear_hidden_size', [128, 256, 512])
    dropout = trial.suggest_categorical('dropout', [0.1, 0.2])
    fc_dropout = trial.suggest_categorical('fc_dropout', [0.1, 0.2])
    head_dropout = trial.suggest_categorical('head_dropout', [0.1, 0.2])
    attn_dropout = trial.suggest_categorical('attn_dropout', [0.1, 0.2])
    patch_len = trial.suggest_int('patch_len', 1, 4)
    patch_len = 16*patch_len
    stride = trial.suggest_int('stride', 1, 4)
    stride = 8*stride
    revin = trial.suggest_categorical('revin', [True, False])
    learning_rate = trial.suggest_int('learning_rate', 1, 5)
    learning_rate = 10**(-learning_rate)
    max_steps = trial.suggest_int('max_steps', 100, 2000)

    models = [PatchTST(h=horizon,
                       input_size=input_size,
                       encoder_layers=encoder_layers,
                       n_heads=n_heads,
                       hidden_size=hidden_size,
                       linear_hidden_size=linear_hidden_size,
                       dropout=dropout,
                       fc_dropout=fc_dropout,
                       head_dropout=head_dropout,
                       attn_dropout=attn_dropout,
                       patch_len=patch_len,
                       stride=stride,
                       revin=revin,
                       learning_rate=learning_rate,
                       max_steps=max_steps,
                       logger=False)
                 ]
    model = NeuralForecast(models=models, freq='D')

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for station in station_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = watercourse_by_stations[station][:-test_len]

        # Rename the columns (library wants to have specific names)
        y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

        # Only keep these 3 columns
        y = y[['ds', 'y', 'unique_id']]

        # Fit the model
        model.fit(y[:-day_len])

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            # Predict
            forecast = model.predict(df=y[:-i])

            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast['PatchTST'].values[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(y['y'][-day_len:], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing on multiple stations

In [ ]:
# List of station used for testing
station_list = ['2530', '2620', '4200', '4230', '4270', '4515', '4520', '4570', '4575', '5040', '5078', '5330', '5425', '5500', '6060', '6068', '6200', '6220', '6300', '6340', '8454', '8565']

In [ ]:
# Cast the stations to int
for i in range(len(station_list)):
    station_list[i] = int(station_list[i])

In [ ]:
'''{'input_size': 19,
 'encoder_layers': 2,
 'n_heads': 2,
 'hidden_size': 256,
 'linear_hidden_size': 256,
 'dropout': 0.1,
 'fc_dropout': 0.1,
 'head_dropout': 0.2,
 'attn_dropout': 0.1,
 'patch_len': 3,
 'stride': 3,
 'revin': False,
 'learning_rate': 3,
 'max_steps': 609}'''

In [ ]:
horizon = 5 # prediction horizon
day_len = 200 # number of days to forecast

model = PatchTST(h=horizon,
                 input_size=19,
                 encoder_layers=4,
                 n_heads=16,
                 hidden_size=256,
                 linear_hidden_size=256,
                 dropout=0.1,
                 fc_dropout=0.1,
                 head_dropout=0.2,
                 attn_dropout=0.1,
                 patch_len=48,
                 stride=24,
                 revin=False,
                 learning_rate=1e-3,
                 max_steps=609,
                 logger=False)

nf = NeuralForecast(
    models=[model],
    freq='D'
)

# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for station in station_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = watercourse_by_stations[station]

    # Rename the columns (library wants to have specific names)
    y = y.rename(columns={'date':'ds', 'level_diff':'y', 'station_id':'unique_id'})

    # Only keep these 3 columns
    y = y[['ds', 'y', 'unique_id']]

    # Fit the model
    nf.fit(y[:-day_len])

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        # Predict
        forecast = nf.predict(df=y[:-i])

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast['PatchTST'].values[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-200:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(watercourse_by_stations[station]['level_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(watercourse_by_stations[station]['date'][-200:], watercourse_by_stations[station]['level_diff'][-200:], color="royalblue", label="true data")
plt.plot(watercourse_by_stations[station]['date'][-day_len:], predictions[2], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/patchtst/patchtst-surface-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/patchtst/patchtst-surface-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(station_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/patchtst/patchtst-surface-water-r2-stations.joblib')

### **4** DeepAR

#### **4.1** Ground water data

##### Quick test 

In [ ]:
#%pip install pytorch-lightning
#%pip install gluonts
#%pip install lightning

In [ ]:
# Read the dataset
aquifer_by_stations = joblib.load('aquifer_by_stations.joblib')

In [ ]:
aquifer = aquifer_by_stations[85065]

In [ ]:
horizon = 5

train_ds = PandasDataset.from_long_dataframe(aquifer[:-horizon], target='altitude_diff', item_id='station_id', 
                                       timestamp='date', freq='D')

In [ ]:
estimator = DeepAREstimator(freq='D', prediction_length=horizon, num_layers=3, trainer_kwargs={'accelerator': 'gpu', 'max_epochs':30, 'logger': False})

predictor = estimator.train(train_ds)

pred = list(predictor.predict(train_ds))

prediction = pred[0].samples.mean(axis=0)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer['date'][-170:], aquifer['altitude_diff'][-170:], color="royalblue", label="True data")
plt.plot(aquifer['date'][-horizon:], prediction, color="tomato", label="Prediction")
plt.grid()
plt.legend()
plt.show()

##### Hyperparameter tuning

In [ ]:
# Define the horizon, day_len (number of predicted days), test_len (number of days used for final testing)
horizon = 5
day_len = 100
test_len = 200

In [ ]:
# Stations to test
aquifers_list = [85065, 85064]

In [ ]:
# Define the function which contains parameters to tune and the model

def objective(trial):
    num_layers = trial.suggest_int('num_layers', 1, 4)
    hidden_size = trial.suggest_int('hidden_size', 20, 200)
    context_length = trial.suggest_int('context_length', 5, 20)
    lr = trial.suggest_int('lr', 1, 5)
    lr = 10**(-lr)
    weight_decay = trial.suggest_int('weight_decay', 7, 9)
    weight_decay = 10**(-weight_decay)
    dropout_rate = trial.suggest_int('dropout_rate', 1, 3)
    dropout_rate = dropout_rate*(1e-1)
    max_epochs = trial.suggest_int('max_epochs', 10, 100)

    model = DeepAREstimator(prediction_length=horizon,
                     freq='D',
                     trainer_kwargs={'accelerator': 'gpu', 'max_epochs': max_epochs, 'logger': False},
                     num_layers=num_layers,
                     hidden_size=hidden_size,
                     context_length=context_length,
                     lr=lr,
                     weight_decay=weight_decay,
                     dropout_rate=dropout_rate)
    

    # List for r2 results for different prediction horizons
    r2_scores = [[] for _ in range(horizon)]
    
    for aquifer in aquifers_list:
        # List for storing the predictions
        predictions = [[] for _ in range(5)]

        # Get the dataset for the aquifer
        y = aquifer_by_stations[aquifer][:-test_len]

        # Change to TimeSeries format (required by the library)
        y_temp = PandasDataset.from_long_dataframe(y[:-day_len], target='altitude_diff', item_id='station_id', 
                                                timestamp='date', freq='D')

        # Fit the model
        predictor = model.train(y_temp)

        # Iterate from day_len days before the end, to the last day
        for i in range(day_len + (horizon-1), 0, -1):
            
            y_temp = PandasDataset.from_long_dataframe(y[:-i], target='altitude_diff', item_id='station_id', 
                                                            timestamp='date', freq='D')
            
            # Predict
            forecast = list(predictor.predict(y_temp))
            
            forecast = forecast[0].samples.mean(axis=0)


            # Store the results for every prediction horizon separately
            for i in range(horizon):
                predictions[i].append(forecast[i])
        
        # Clean up the results
        predictions[0] = predictions[0][-day_len:]
        predictions[1] = predictions[1][3:-1]
        predictions[2] = predictions[2][2:-2]
        predictions[3] = predictions[3][1:-3]
        predictions[4] = predictions[4][0:-4]

        # Calculate the r2 scores and store them in a list
        for i in range(horizon):
            r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-(day_len+test_len):-test_len], predictions[i]))
    
    # Calculate the average r2 score
    r2_average =  []
    
    for i in range(5):
        r2_average.append(np.mean(r2_scores[i]))

    # Set the loss as average of average r2 scores for different prediction horizons
    loss = np.mean(r2_average)

    return loss

In [ ]:
# Run the optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

In [ ]:
study.best_params

In [ ]:
study.best_value

##### Testing multiple stations

In [ ]:
aquifers_list = [85065, 85064]

In [ ]:
'''{'num_layers': 1,
 'hidden_size': 158,
 'context_length': 15,
 'lr': 3,
 'weight_decay': 9,
 'dropout_rate': 1,
 'max_epochs': 14}'''
# To Do !!! test the model with these hyperparameters

In [ ]:
horizon = 5 # prediction horizon
day_len = 365 # number of days to forecast

# Set the model parameters
model = DeepAREstimator(prediction_length=horizon,
                 freq='D',
                 trainer_kwargs={'accelerator': 'gpu', 'max_epochs': 14, 'logger': False},
                 num_layers=1,
                 hidden_size=158,
                 context_length=15,
                 lr=1e-3,
                 weight_decay=9,
                 dropout_rate=1)


# List for r2 results for different prediction horizons
r2_scores = [[] for _ in range(horizon)]

for aquifer in aquifers_list:
    # List for storing the predictions
    predictions = [[] for _ in range(5)]

    # Get the dataset for the aquifer
    y = aquifer_by_stations[aquifer]

    # Change to TimeSeries format (required by the library)
    y_temp = PandasDataset.from_long_dataframe(y[:-day_len], target='altitude_diff', item_id='station_id', 
                                                timestamp='date', freq='D')
    # Fit the model
    predictor = model.train(y_temp)

    # Iterate from day_len days before the end, to the last day
    for i in range(day_len + (horizon-1), 0, -1):
        
        y_temp = PandasDataset.from_long_dataframe(y[:-i], target='altitude_diff', item_id='station_id', 
                                                        timestamp='date', freq='D')

        # Predict
        forecast = list(predictor.predict(y_temp))
        
        forecast = forecast[0].samples.mean(axis=0)

        # Store the results for every prediction horizon separately
        for i in range(horizon):
            predictions[i].append(forecast[i])
    
    # Clean up the results
    predictions[0] = predictions[0][-day_len:]
    predictions[1] = predictions[1][3:-1]
    predictions[2] = predictions[2][2:-2]
    predictions[3] = predictions[3][1:-3]
    predictions[4] = predictions[4][0:-4]

    # Calculate the r2 scores and store them in a list
    for i in range(horizon):
        r2_scores[i].append(r2_score(aquifer_by_stations[aquifer]['altitude_diff'][-day_len:], predictions[i]))

In [ ]:
# Calculate the average r2 score
r2_average =  []
std_dev = []

for i in range(5):
    r2_average.append(np.mean(r2_scores[i]))
    std_dev.append(np.std(r2_scores[i]))

In [ ]:
r2_average

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(aquifer_by_stations[aquifer]['date'][-200:], aquifer_by_stations[aquifer]['altitude_diff'][-200:], color="royalblue", label="true data")
plt.plot(aquifer_by_stations[aquifer]['date'][-day_len:], predictions[0], color="tomato", label="forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Save the average r2_scores
with open('../reports/n-hits/n-hits-ground-water-r2.txt', 'w') as file:
    for item in r2_average:
        file.write(f"{item}\n")

In [ ]:
# Save the standard deviations
with open('../reports/n-hits/n-hits-ground-water-std-dev.txt', 'w') as file:
    for item in std_dev:
        file.write(f"{item}\n")

In [ ]:
# Transpose the r2_scores list
r2_scores_transposed = [list(x) for x in zip(*r2_scores)]
# Pair up the stations with their r2_scores and store them in a dictionary
scores = dict(zip(aquifers_list, r2_scores_transposed))
scores

In [ ]:
# Sort them by the value in r2_scores[0]
scores_sorted = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1][0])}
scores_sorted

In [ ]:
# Save the r2_scores
joblib.dump(scores_sorted, '../reports/n-hits/n-hits-ground-water-r2-stations.joblib')

#### **4.2** Watercourse data